<a href="https://colab.research.google.com/github/gabsgeografo/InteligenciaArtificialMestrado/blob/main/Data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das bibliotecas

In [ ]:
import os
import numpy as np
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import tensorflow
import random
from glob import glob
from imageio import mimread
from google.colab.patches import cv2_imshow
tensorflow.__version__

'2.8.2'

Conexão com Google Drive onde estão as imagens

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Carregamento da base de dados

Criar a pasta IA no colab para salvara as imagens que estão no google Drive

In [ ]:
def criar_diretorio(caminho):
  if not os.path.exists(caminho):
    os.makedirs(caminho)

criar_diretorio('IA')    

!cp -r /content/gdrive/Othercomputers/Meu\ laptop/Relatorio/Originais /content/IA/images

Verificação se os arquivos estão na pasta. Foram disponibilizados 29 imagens, 1 delas possui ruídos (traços amarelos na imagem) e foi removida da base de dados. Portanto temos 28 imagens.

In [ ]:
import os
initial_count = 0
dir = "/content/IA/images"
for path in os.listdir(dir):
    if os.path.isfile(os.path.join(dir, path)):
        initial_count += 1
print(initial_count)

28


Para evitar o overffting, deve-se gerar mais imagens. Com 28 imagens não é possivel gerar um bom modelo de classificação.

A primeira tratativa foi remover uma tarja localizada na parte de baixo das imagens.

A segunda foi utilizada a técnica de Transposição em cada imagem gerando 56 imagens.



In [ ]:
from albumentations.augmentations.crops.transforms import RandomCrop
from skimage import color
from skimage import io
from albumentations import HorizontalFlip, VerticalFlip, ElasticTransform, GridDistortion, OpticalDistortion, CoarseDropout, RandomBrightnessContrast, Transpose

def novas_imagens(imagens, dir_salvo, img_altura=512, img_largura=512, augmentation=True, total=True):
  for idx, (x) in tqdm(enumerate(imagens), total = len(imagens)):
    nome = x.split('/')[-1].split('.')[0]
    #print(nome)

    x = cv2.imread(x)

    if augmentation:
      if total:
        aug = HorizontalFlip(p = 1.0)
        augmentation = aug(image = x)
        x1 = augmentation['image']

        aug = VerticalFlip(p = 1.0)
        augmentation = aug(image = x)
        x2 = augmentation['image']

        X = [x, x1, x2]
      else:
        aug = Transpose(always_apply=False, p=1.0)
        augmentation = aug(image = x)
        x4 = augmentation['image'] 
        X = [x,x4]
    else:
      X = [x]

    indice = 0

    for img in X:
      img = cv2.resize(img, (img_largura, img_altura))

      if len(X) == 1:
        tmp_img_nome = f"{nome}.png"
      else:
        tmp_img_nome = f"{nome}_{indice}.png"

      path_imagem = os.path.join(dir_salvo, tmp_img_nome)
     
      cv2.imwrite(path_imagem, img)      

      indice += 1

In [ ]:
# for google colab use cv2_imshow instead
# change x, y, h ,w to the values you are happy with

img_altura = 510
img_largura = 512

def cortarImagem(arquivo,destino,total = True):
  import cv2
  from google.colab.patches import cv2_imshow
  import imageio

  file_path = arquivo

  file_path = os.path.splitext(file_path)[0]
  file_name = file_path.split('/')[-1]

  img = cv2.imread(arquivo)

  
  alt = 510
  lar = 512

  imagem = img[0:510, 0:512]
  imageio.imwrite(destino + file_name + '_1.png', imagem)

  if total:
    imagem = img[0:int(alt/2), 0:int(lar/2)]
    imageio.imwrite(destino + file_name +'_2.png', imagem)

    imagem = img[int(alt/2):alt, 0:int(lar/2)]
    imageio.imwrite(destino + file_name + '_3.png', imagem)

    imagem = img[0:int(alt/2), int(lar/2):lar]
    imageio.imwrite(destino + file_name +  '_4.png', imagem)

    imagem = img[int(alt/2):alt, int(lar/2):lar]
    imageio.imwrite(destino + file_name +  '_5.png', imagem)

    imagem = img[127:382, 127:382]
    imageio.imwrite(destino + file_name +  '_6.png', imagem)

In [ ]:
#Remover as Tarjas
criar_diretorio('IA/SemTarja')

from re import X
import os

pasta = '/content/IA/images'

caminhos = [os.path.join(pasta, nome) for nome in os.listdir(pasta)]
arquivos = [arq for arq in caminhos if os.path.isfile(arq)]
jpgs = [arq for arq in arquivos if arq.lower().endswith(".tif")]

for x in jpgs:
  cortarImagem(x,'/content/IA/SemTarja/',False) 

In [ ]:
import os
initial_count = 0
dir = "/content/IA/SemTarja"
for path in os.listdir(dir):
    if os.path.isfile(os.path.join(dir, path)):
        initial_count += 1
print(initial_count)

28


In [ ]:
def carregar_imagens(caminho):
  X_train = sorted(glob(os.path.join(caminho, "SemTarja", "*.png")))
  
  return (X_train)

path_imagen = "IA"
X_train = carregar_imagens(path_imagen) 

len(X_train)

28

In [ ]:
criar_diretorio('IA/Transposta')
#Transposição
novas_imagens(X_train, 'IA/Transposta/', augmentation=True, total=False)


100%|██████████| 28/28 [00:01<00:00, 20.54it/s]


In [ ]:
import os
initial_count = 0
dir = "/content/IA/Transposta"
for path in os.listdir(dir):
    if os.path.isfile(os.path.join(dir, path)):
        initial_count += 1
print(initial_count)

56


Neste terceira etapa as imagens transpostas serão cortadas gerando outras 5 imagens.

1.   Quanto superior esquerdo
2.   Quanto superior direito
3.   Quanto inferior esquerdo
4.   Quanto inferior direito
5.   Um quadrado, com as mesmas dimensões dos cortes anteiores, no centro.

Com isso foram geradas: 336 imagens (5 geradas + 1 original = 6) (6 x 56 = 336)

In [ ]:
#Cortar as imagens sem Tarja gerando as novas imagens
criar_diretorio('IA/Geradas')

from re import X
import os
pasta = '/content/IA/Transposta/'

caminhos = [os.path.join(pasta, nome) for nome in os.listdir(pasta)]
arquivos = [arq for arq in caminhos if os.path.isfile(arq)]
jpgs = [arq for arq in arquivos if arq.lower().endswith(".png")]

for x in jpgs:
  cortarImagem(x,'/content/IA/Geradas/',True)  

Conferência da quantidade de Imagens

In [ ]:
import os
initial_count = 0
dir = "/content/IA/Geradas"
for path in os.listdir(dir):
    if os.path.isfile(os.path.join(dir, path)):
        initial_count += 1
print(initial_count)

336


In [ ]:
!cp -r /content/IA/SemTarja/ /content/gdrive/Othercomputers/Meu\ laptop/Relatorio/SemTarja
!cp -r /content/IA/Geradas/ /content/gdrive/Othercomputers/Meu\ laptop/Relatorio/Geradas

As demais técnicas de data augmentation foram aplicadas nas imagens geradas (após os cortes).

1.   Gerar verticalmente
2.   Gerar horizontalmente

Com isso foram geradas 1008 imagens (2 geradas + 1 originais = 3) (3 x 336 = 1008) 



Gera um array com as imagens Geradas. Essas imagens serão usadas no processo de data augmentation

In [ ]:
def carregar_imagens(caminho):
  X_train = sorted(glob(os.path.join(caminho, "Geradas", "*.png")))
  
  return (X_train)

path_imagen = "IA"
X_train = carregar_imagens(path_imagen) 

len(X_train)

336

In [ ]:
criar_diretorio('IA/Novas')

novas_imagens(X_train, 'IA/Novas/', augmentation=True, total=True)

100%|██████████| 336/336 [00:19<00:00, 17.35it/s]


In [ ]:
import os
initial_count = 0
dir = "/content/IA/Novas"
for path in os.listdir(dir):
    if os.path.isfile(os.path.join(dir, path)):
        initial_count += 1
print(initial_count)

1008


Imagens geradas e salvas no Google Drive para serem usadas na redes que serão motnadas

In [ ]:
!cp -r /content/IA/ /content/gdrive/Othercomputers/Meu\ laptop/Relatorio/Novas